In [ ]:
import bs4
import pandas as pd
import requests

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get("https://www.newsweek.com/newsfeed?page=1", headers=headers)
response

In [ ]:
soup = bs4.BeautifulSoup(response.text)
g_div = soup.find_all("article", class_="l3")
g_div[0]

In [ ]:
#Find "category"
cat = g_div[0].find("div", class_="category")

#Get only the text
cat.text

In [ ]:
# Find url
g_div[0].a['href']

for i in g_div:
    print( "Found the URL:", i.a['href'])

In [ ]:
# Get the title of the article in the first box
g_div[0].find("h3").text

In [ ]:
# Find the div with the "summary" class
sum = g_div[0].find("div", class_="summary")
sum.text

In [ ]:
categories = []
urls = []
titles = []
summaries = []

for div in g_div:
    try:
        url = div.a['href']
        urls.append(url)
    except:
        pass

    try:
        cat = div.find("div", class_="category")
        categories.append(cat.text)
    except:
        pass

    try:
        title = div.find("h3").text
        titles.append(title)
    except:
        pass

    try:
        sum = div.find("div", class_="summary")
        summaries.append(sum.text)
    except:
        pass




In [ ]:
df = pd.DataFrame({
    'Category':categories,
    'URL':urls,
    'Title':titles,
    'Summary':summaries
})
df

In [ ]:
# Loop through 50 pages of listings and store category, URL, title, and summary in lists
cat1=[]
url1=[]
title1=[]
sum1=[]

base_url = "https://www.newsweek.com/newsfeed"

for i in range(1,51):
    url = base_url+ "?page=" + str(i)
    #print(url)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(res.text)
    g_div = soup.find_all("article", class_="l3")
    for div in g_div:
        try:
            url = div.a['href']
            url1.append(url)
        except:
            pass

        try:
            cat = div.find("div", class_="category")
            cat1.append(cat.text)
        except:
            pass

        try:
            title = div.find("h3").text
            title1.append(title)
        except:
            pass

        try:
            sum = div.find("div", class_="summary")
            sum1.append(sum.text)
        except:
            pass

  

In [ ]:
df = pd.DataFrame({
    'Category':cat1,
    'URL':url1,
    'Title':title1,
    'Summary':sum1
})

df

In [ ]:
df.to_csv("newsweek.csv")

## Scraping one whole article

In [ ]:
# Get the URL for the first article
url_col = "https://newsweek.com" + df.iloc[1][1]
url_col

In [ ]:
resp = requests.get(url_col, headers=headers)
soup1 = bs4.BeautifulSoup(resp.text)
soup1

In [ ]:
h_div = soup1.find_all("div", class_='article-body v_text paywall')

In [ ]:
all_p = soup1.find_all('p')

In [ ]:
# Join all tags together
article = all_p[0].text + " "

for i in range(1,len(all_p)):
    article = article + all_p[i].text + " "

article

In [ ]:
# Scrape the articles found in the url column

articles = []
for i in range(0,len(df)):
    url_col = "https://newsweek.com" + df.iloc[i][1]
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    resp = requests.get(url_col, headers=headers)
    soup1 = bs4.BeautifulSoup(resp.text)
    h_div = soup1.find_all("div", class_='article-body v_text paywall')
    all_p = soup1.find_all('p')
    
    article = all_p[0].text + " "

    for i in range(1,len(all_p)):
        try:
            article = article + all_p[i].text + " "
        except:
            pass
    articles.append(article)

df['Full_text'] = articles

In [ ]:
df.to_csv("newsweek2.csv")